In [22]:
! pip install llama-index == 0.11.13 
! pip install llama-index-llms-ollama
! pip install llama-index-embeddings-ollama

zsh:1: = not found


In [32]:
import os 
import logging
import sys

from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_host = os.getenv("OLLAMA_API", "http://localhost:11434")
ollama_model = os.getenv("OLLAMA_MODEL_LLAMA", "llama3.2:latest")
logging.basicConfig(level=logging.DEBUG, stream=sys.stdout)
logger = logging.getLogger(__name__)
logger.debug(f"ollama_host: {ollama_host}")
logger.debug(f"ollama_model: {ollama_model}")
llm = Ollama(model=ollama_model, base_url=ollama_host)
embedding = OllamaEmbedding(model_name=ollama_model, base_url=ollama_host)
Settings.embed_model = embedding
Settings.llm = llm

DEBUG:__main__:ollama_host: http://localhost:11434
DEBUG:__main__:ollama_model: llama3.2:latest
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/tju/.pyenv/versions/3.11.10/lib/python3.11/site-packages/certifi/cacert.pem'
DEBUG:httpx:load_ssl_context verify=True cert=None trust_env=True http2=False
DEBUG:httpx:load_verify_locations cafile='/Users/tju/.pyenv/versions/3.11.10/lib/python3.11/site-packages/certifi/cacert.pem'


In [33]:
from llama_index.core import SimpleDirectoryReader
documents = SimpleDirectoryReader(input_dir="./ch2/files").load_data()

DEBUG:llama_index.core.readers.file.base:> [SimpleDirectoryReader] Total files added: 2
DEBUG:fsspec.local:open file: /Users/tju/Workspace/books/llama-index-practise/ch2/files/sample_document1.txt
DEBUG:fsspec.local:open file: /Users/tju/Workspace/books/llama-index-practise/ch2/files/sample_document2.txt


> 使用Pipeline喂入文档

In [ ]:
from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.extractors import TitleExtractor
import asyncio, nest_asyncio

pipeline = IngestionPipeline(
    transformations = [
        SentenceSplitter(chunk_size=1000, chunk_overlap=200),
        TitleExtractor(),
        embedding
    ]
)
# Please use nest_asyncio.apply() to allow nested event loops
# nodes = nest_asyncio.apply(pipeline.run(documents))
asyncio.run(pipeline.run(documents))


> 使用VectorStoreIndex喂入文档

In [34]:
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(llm=llm)
resp = query_engine.query("summarize each documents in a few sentences")
logger.debug(f"resp: {resp}")


DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: In ancient Rome, the city of Rome itself was th...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Many people consider dogs to be their loyal com...
DEBUG:httpcore.connection:connect_tcp.started host='localhost' port=11434 local_address=None timeout=None socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x16c1efc10>
DEBUG:httpcore.http11:send_request_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_headers.complete
DEBUG:httpcore.http11:send_request_body.started request=<Request [b'POST']>
DEBUG:httpcore.http11:send_request_body.complete
DEBUG:httpcore.http11:receive_response_headers.started request=<Request [b'POST']>
DEBUG:httpcore.http11:receive_response_headers.complete return_value=(b'HTTP/1.1', 200, b'OK', [(b'Content-Type', b'application/json; charset=utf-8'), (b'Date', b'Mon, 18 Nov 2024 12:01:01 G